# FNC 6.6 Multistep Methods

For each method (AM3, AB3, BD4), compute the **leading term** of the local truncation error using the series approach suggested by Eq. (6.6.5). Then optionally verify the order numerically on a smooth test function.

We express a $k$-step method as
$$\sum_j \alpha_j u_{n+j} - h\sum_j \beta_j f_{n+j} = 0,$$
and define the single–step **defect**
$$R(h)=\sum_j \alpha_j u(t_n+jh) - h\sum_j \beta_j u'(t_n+jh),\qquad \tau(h)=R(h)/h.$$


In [1]:

from fractions import Fraction
from collections import defaultdict
from math import factorial

def lte_series(alpha, beta, max_order=6):
    """Return series dict for tau(h)=R(h)/h.
    alpha: list[(j,alpha_j)], beta: list[(j,beta_j)], with integer shifts j.
    Output: {m: {r: coeff}} such that tau(h) = sum_{m>=0} h^m * sum_r coeff * u^{(r)}(t_n).
    """
    alpha = [(int(j), Fraction(a).limit_denominator()) for j,a in alpha]
    beta  = [(int(j), Fraction(b).limit_denominator()) for j,b in beta]
    # A_k = sum alpha_j j^k/k!,  B_k = sum beta_j j^k/k!
    A = [sum(a * Fraction(j**k, factorial(k)) for j,a in alpha) for k in range(max_order+1)]
    B = [sum(b * Fraction(j**k, factorial(k)) for j,b in beta)  for k in range(max_order+1)]
    # tau(h) = sum_{k>=0} c_k h^{k-1} A_k - sum_{k>=0} c_{k+1} h^{k} B_k
    series = defaultdict(lambda: defaultdict(Fraction))
    for k in range(max_order+1):
        if k-1 >= 0:
            series[k-1][k] += A[k]
    for k in range(max_order+1):
        series[k][k+1] -= B[k]
    # trim zeros
    out = {}
    for m in sorted(series):
        if m < 0: continue
        row = {r:series[m][r] for r in series[m] if series[m][r] != 0}
        if row: out[m] = row
    return out

def leading_term(series):
    """Pick the first nonzero term (smallest m). Return (m, r, coeff)."""
    for m in sorted(series):
        r = sorted(series[m])[0]
        return m, r, series[m][r]
    return None

def pretty_series(series, upto=5):
    lines = []
    for m in sorted(series):
        if m>upto: break
        terms = " + ".join([f"({series[m][r]})*u^{({r})}" for r in sorted(series[m])])
        lines.append(f"h^{m} * [ {terms} ]")
    return "\n".join(lines)


## Define the three methods (coefficients)

In [2]:

# AM3: u_{n+1} - u_n = h * (-1/12 f_{n-1} + 2/3 f_n + 5/12 f_{n+1})
AM3 = ([(0,-1),(1,1)], [(-1, -Fraction(1,12)), (0, Fraction(2,3)), (1, Fraction(5,12))])

# AB3: u_{n+1} - u_n = h * (23/12 f_n - 16/12 f_{n-1} + 5/12 f_{n-2})
AB3 = ([(0,-1),(1,1)], [(0, Fraction(23,12)), (-1, -Fraction(16,12)), (-2, Fraction(5,12))])

# BD4 (BDF4): 25/12 u_{n+1} - 4 u_n + 3 u_{n-1} - 4/3 u_{n-2} + 1/4 u_{n-3} = h f_{n+1}
BD4 = ([(1, Fraction(25,12)), (0, -4), (-1, 3), (-2, -Fraction(4,3)), (-3, Fraction(1,4))], [(1,1)])

methods = {"AM3": AM3, "AB3": AB3, "BD4": BD4}
for name, (alpha,beta) in methods.items():
    ser = lte_series(alpha, beta, max_order=6)
    m,r,c = leading_term(ser)
    print(f"{name}: tau(h) leading term  = ({c}) * h^{m} * u^({r})  "
          f"=> defect R(h)=h*tau ~ ({c}) * h^{m+1} * u^({r})")
    print(pretty_series(ser, upto=m), "\n")


AM3: tau(h) leading term  = (-1/24) * h^3 * u^(4)  => defect R(h)=h*tau ~ (-1/24) * h^4 * u^(4)
h^3 * [ (-1/24)*u^{4} ] 

AB3: tau(h) leading term  = (3/8) * h^3 * u^(4)  => defect R(h)=h*tau ~ (3/8) * h^4 * u^(4)
h^3 * [ (3/8)*u^{4} ] 

BD4: tau(h) leading term  = (-1/5) * h^4 * u^(5)  => defect R(h)=h*tau ~ (-1/5) * h^5 * u^(5)
h^4 * [ (-1/5)*u^{5} ] 




**Reference values (should match the printout):**  
- AM3: $\tau(h) \sim -\tfrac{1}{24} h^{3} u^{(4)}$, i.e. $R(h) \sim -\tfrac{1}{24} h^{4} u^{(4)}$ (order 3).  
- AB3: $\tau(h) \sim +\tfrac{3}{8} h^{3} u^{(4)}$, i.e. $R(h) \sim +\tfrac{3}{8} h^{4} u^{(4)}$ (order 3).  
- BD4: $\tau(h) \sim -\tfrac{1}{5} h^{4} u^{(5)}$, i.e. $R(h) \sim -\tfrac{1}{5} h^{5} u^{(5)}$ (order 4).


## Optional: numerical check of the observed order (defect $R(h)$)

In [3]:

import numpy as np, math

def defect(alpha, beta, u, up, h):
    t0 = 0.0
    R = 0.0
    for j,a in alpha:
        R += float(a) * u(t0 + j*h)
    S = 0.0
    for j,b in beta:
        S += float(b) * up(t0 + j*h)
    return abs(R - h*S)

def check_order(name, alpha, beta, u, up, hs):
    errs = np.array([defect(alpha,beta,u,up,h) for h in hs])
    slope = np.polyfit(np.log(hs), np.log(errs), 1)[0]
    return slope

u = lambda t: math.sin(t)
up = lambda t: math.cos(t)

hs = np.array([0.8, 0.4, 0.2, 0.1, 0.05])
for name,(alpha,beta) in methods.items():
    p = check_order(name, alpha,beta,u,up,hs)
    print(f"{name}: observed R(h) ~ h^{p:.3f}  (expected ~ h^{ {'AM3':4, 'AB3':4, 'BD4':5}[name] })")


AM3: observed R(h) ~ h^4.990  (expected ~ h^4)
AB3: observed R(h) ~ h^4.966  (expected ~ h^4)
BD4: observed R(h) ~ h^4.920  (expected ~ h^5)
